# Read Me

This project uses linkedin to send automatic messages in order to find a job faster

### Goals of the project
- Finding recruiters and people working in the industry i want and send them appropriate messages (adding the possibility to schedule a meeting with me on my personnal calendar ?
- Apply to the positions that fits my criteria (only working for simplified job apply imo)


### TODO
- Find optimal recruiter, find out how the localization filter works
- Work on finding optimal job post to send message to the recruiter from the job post


### pipeline to dm recruiters from job post
- get list of job throught the job search bar from linkedin (still can optimize this throught better keywords and location not working atm)
- iterate throught each job, if there is a recruiter who writed the job post, get it's id from selenium scrap
- send a connection invite proposing to have a quick chat about the role (add few keywords ?)
- connect the response to a collab excel to save the info


In [1]:
from linkedin_api import Linkedin
from linkedin_api.utils.helpers import get_id_from_urn

In [2]:
def open_file(filepath:str)-> str:
    '''Opens and reads a file'''
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
    
    
def get_credential(path):
    raw_data = open_file(path)
    data = raw_data.split(",")
    return [data[0][2:-1], data[1][1:-2]]

    

In [3]:
# Authenticate using any Linkedin account credentials


test_credential = get_credential('test_credential.txt')
true_credential = get_credential('linkedin_username.txt')

test_api = Linkedin(test_credential[0], test_credential[0])
real_api = Linkedin(true_credential[0], true_credential[0])


# GET a profile
#profile = api.get_profile('billy-g')

# GET a profiles contact info
#contact_info = api.get_profile_contact_info('billy-g')

# GET 1st degree connections of a given profile    USE URN ID
#connections = api.get_profile_connections('1234asc12304')


# First part : finding recruiters and sending them invite with personnal messages

- First dichotomy, from their summary we try to define if they are in the quant finance or in the tech sector to send the more accurate message

- Second one, if they are based in France we send our message in French

- To get better results we could developp a rating on how much we would be interested in each profile

In [74]:
#message_data = "I'm a graduate from an MSc in Statistics from Université PSL in Paris, " \
#    + "after a year of experience as a Data Scientist, I am looking for" \
#    + "a new opportunity in this sector."   

#message_quant = "I'm a graduate from an MSc in Statistics and Quantitative Finance from Université " \
#    + "PSL in Paris, after a year of experience as a Data Scientist, I am now looking to" \
#    + "work as a Quant."  


message_quant = "graduate from an MSc in Statistics and Quantitative Finance. An experience as a data scientist, looking for an opportunity as a quant"
message_data = "graduate from an MSc in Statistics. An experience as a data scientist, looking for a new opportunity, data-driven."
    
end_of_message = "Feel free to contact me if you see a possible fit !"


data_recruiter = 0
quant_recruiter = 0
na_recruiter = 0

connections = []


regions_known = {'USA' : '103644278', 'France' : '105015875', 'UK' : '101165590', 'Holand' : '102890719'}


['103644278', '105015875', '101165590', '102890719']

In [ ]:
len(message_data)

In [80]:
def is_profile_data_or_finance(profile, data_keywords = ['tech','data','engineer', 'analyst', 'analytics'] , quant_finance_keywords = ['quantitative', 'finance', 'fintech', 'financial', 'hft', 'hedge', 'asset']):
    
    count_data = 0
    count_quant = 0
    
    # Some recruiters don't have a summary, in this case we take the title
    try:
        summary = profile['summary'].lower()

    except KeyError:
        summary = profile['headline'].lower()
        
    summary = summary.split(" ")
    
    for word in summary:
        
        if(word in data_keywords):
            count_data += 1
            
        if(word in quant_finance_keywords):
            count_quant += 1
            
    if(count_quant > count_data):
        return 'finance'
    
    if(count_data > 0):
        return 'data'
    
    return 'NA'


def is_profile_fetched_french(profile, france_keywords = ['france', 'paris', 'île-de-france']):
    
    try:
        location = profile['location'].lower()
        
    except KeyError:
        return False
    
    for keyword in france_keywords:
        if(keyword in location):
            return True
    
    return False


def get_first_name_fetched_profile(profile):
    
    try:
        first_name = profile['name'].split(" ")[0]
    
    except:
        return False
    return first_name



def add_easy_personnalized_connection(current_recruiter, api, message_quant = message_quant,
                                 message_data = message_data, end_of_message = end_of_message):

    message = ""

    first_name = get_first_name_fetched_profile(current_recruiter)
    is_french = is_profile_fetched_french(current_recruiter)
    current_profile = api.get_profile(current_recruiter['public_id'])
    sector = is_profile_data_or_finance(current_profile)


    if(sector == 'finance'):
        message = "Hello " + first_name + "\n" + message_quant + "\n" + end_of_message

    elif(sector == 'data'):
        message = "Hello " + first_name + "\n" + message_data + "\n" + end_of_message

    else:  
        return "na_recruiter"

    connection = api.add_connection(current_recruiter['public_id'], message)
    
    return [sector, connection]







def get_gpt_message(generic_message, profile, first_name):
        
    discussion = [
        {"role": "system", "content": "You adapt a generic message to a personnalized message saying that I am looking for a new opportunity to headhunters."},
        {"role": "user", "content": f"Here is the generic message : <<{generic_message}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{profile['summary']}>> and his first name : <<{first_name}>>"},
        {"role": "assistant", "content": "What is the expected output ?"},        
        {"role": "user", "content": "Provide it in JSON format with the key : message. No more than 60 words."},
        {"role": "assistant", "content": "What is the expected content?"}, 
    ]


    message_prompt = "Greet the headhunter by the first name given, say that from my experience and "\
    + "qualification he may have opportunities that would fit my profile. "\
    + "Return just the JSON with the key message, nothing else"
    answer, discussion = gpt3_chat(message=message_prompt, messages=discussion)
    
    print(answer)
    answer = extract_only_dict_from_gpt_output(answer)
    print(answer)
    answer = json_to_dict(answer)
    
    return answer["message"]



def add_gpt_personnalized_connection(current_recruiter, api, message_quant = message_quant,
                                 message_data = message_data, end_of_message = end_of_message):

    
    #first_name = get_first_name_fetched_profile(current_recruiter)   sometimes gives false idk why
    print(current_recruiter['name'])
    first_name = current_recruiter['name'].split(" ")[0]
    is_french = is_profile_fetched_french(current_recruiter)
    current_profile = api.get_profile(current_recruiter['public_id'])
    sector = is_profile_data_or_finance(current_profile)


    if(sector == 'finance'):
        generic_message = message_quant

    else:
        generic_message = message_data

    #else:  
    #    return "na_recruiter"
    
    personnalized_message = get_gpt_message(generic_message, current_profile, first_name)
    final_message = personnalized_message
    #final_message = "Hello " + first_name + "\n" + personnalized_message
    
    if(len(final_message) >299):
        final_message = final_message[0:300] 
    
    
    print(final_message)

    connection = api.add_connection(current_recruiter['public_id'], final_message)
    
    return [sector, connection, final_message]



In [17]:
def create_message_for_recruiter_from_profile(recruiter_profile_id, api = test_api):
    
    recruiter = api.get_profile(recruiter_profile_id)
    first_name = recruiter['firstName']
    
    if(first_name):
        
        message = "Hello "+ first_name + ", \n" + "I am a graduate from an Msc in Statistics with an experience " \
                  + "as a Data Scientist and I came across your profile. I am currently looking for a new position." \
                  + " Would you be available for an exchange to see if you might have an opportunity that could " \
                  + "fit with me ?"

    else:
        
        message = "Hello, I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    
    return message

In [63]:
search_quant = test_api.search_people(keywords = 'quantitative finance', keyword_title = 'recruiter')

In [77]:
people_searched = test_api.search_people(keywords = 'tech AI ML', keyword_title = 'recruiter', regions = list(regions_known.values()), limit = 100)

In [78]:
people_searched

[{'urn_id': 'ACoAAAprcIIB6btozi9AgbclmaxMjTx_OHOPKN0',
  'distance': 'DISTANCE_3',
  'public_id': 'henrywylder',
  'tracking_id': '174813314',
  'jobtitle': 'Tech Recruiter - Space / Robotics / AI / ML',
  'location': 'Los Angeles, California, United States',
  'name': 'Henry Wylder'},
 {'urn_id': 'ACoAABnt7vcBdeUyuistQBfMXuUbE4VltV1nqoA',
  'distance': 'DISTANCE_3',
  'public_id': 'shaun-thompson-nicholls-933831101',
  'tracking_id': '435023607',
  'jobtitle': 'Tech Recruiter (ML/AI) @ Meta | Proud Veteran | He/Him',
  'location': 'London, England, United Kingdom',
  'name': 'Shaun Thompson-Nicholls'},
 {'urn_id': 'ACoAAAhop7YBUMH27DaINJ7pKJ8x1ONalNmXANw',
  'distance': 'DISTANCE_3',
  'public_id': 'hadilfilali',
  'tracking_id': '141076406',
  'jobtitle': 'Lead Data Recruiter #AI #DataScience #BigData',
  'location': 'Paris, Île-de-France, France',
  'name': 'Hadil Filali'},
 {'urn_id': 'ACoAAAAGidkBSW6oIpwb-IeXoxxg0FRwaDP6jnM',
  'distance': 'DISTANCE_3',
  'public_id': 'alexwenz',


In [13]:
# Create sheet to see message send to recruiter from people search

mail = "tang.bellec@gmail.com"
#sheet = create_sheet("messages_recruiter_people_search", mail)
sheet = connect_to_sheet("messages_recruiter_people_search")
#add_new_row(sheet, ["recruiter_id", "message", "date", "answer"])

In [38]:
def extract_only_dict_from_gpt_output(gpt_answer):
    in_json = False
    
    for current_char in gpt_answer:
        if(in_json):
            json = json + current_char
            if(current_char == '}'):
                return json
        else:
            if(current_char == '{'):
                json = '{'
                in_json = True

In [82]:
people_searched[10:]

[{'urn_id': 'ACoAAABBv4ABWvc6RsxawBrNDLMvyvdkDEnrPSw',
  'distance': 'DISTANCE_3',
  'public_id': 'kurtbannisterexecutivesearch',
  'tracking_id': '4308864',
  'jobtitle': 'Executive Recruiter in AR/VR, AI/ML, Fintech, Research, Video Games, Consumer Electronics',
  'location': 'London Area, United Kingdom',
  'name': 'Kurt Bannister'},
 {'urn_id': 'ACoAAABhidYBSs4CjXIJW7KVe5YuJF0zlTyDpUc',
  'distance': 'DISTANCE_3',
  'public_id': 'davidgmann',
  'tracking_id': '6392278',
  'jobtitle': 'Lead Recruiter (AI/ML & Technology) - Bridgewater Associates',
  'location': 'Los Angeles Metropolitan Area',
  'name': 'David Mann'},
 {'urn_id': 'ACoAABKu_fgBaem2Utk11glaQ2MqIgO3vaBNoEo',
  'distance': 'DISTANCE_3',
  'public_id': 'peter-molnár-4a0b8788',
  'tracking_id': '313458168',
  'jobtitle': 'Recruiting AI / ML @Metaverse!',
  'location': 'Amsterdam, North Holland, Netherlands',
  'name': 'Peter Molnár'},
 {'urn_id': 'ACoAAAO0AE0BSjBEYiQ6tuEinJJlHXUU74M1WsA',
  'distance': 'DISTANCE_3',
  'pu

In [ ]:
# Pipeline with my test account
connections = []
quant_recruiter = 0
data_recruiter = 0

from datetime import datetime
date_today = datetime.today().strftime('%Y-%m-%d')

for recruiter in people_searched[75:]:
        
    try:
        current_connection = add_gpt_personnalized_connection(recruiter, real_api)
        add_new_row(sheet, [recruiter['public_id'], current_connection[2], date_today, ""])
        print('no key error')
        
    except KeyError:    # Happends when no summary or profile can't be found
        print('key error')
        continue
        
        
    connections.append(current_connection)
    
    if(current_connection[0] == 'finance'):
        quant_recruiter += 1
    if(current_connection[0] == 'data'):
        data_recruiter += 1
    else:
        na_recruiter += 1

Olivia Barreto
Here's the JSON message you requested:

```json
{
  "message": "Hi Olivia! With my MSc in Statistics and experience in data science, I believe you may have opportunities that would fit my profile. Can we connect to discuss further?"
}
```
{
  "message": "Hi Olivia! With my MSc in Statistics and experience in data science, I believe you may have opportunities that would fit my profile. Can we connect to discuss further?"
}
Hi Olivia! With my MSc in Statistics and experience in data science, I believe you may have opportunities that would fit my profile. Can we connect to discuss further?
no key error
Adriel Cohen
Here is the requested JSON format message:

```json
{
    "message": "Hello Adriel, as a graduate from an MSc in Statistics with an experience in data science, I believe that you may have opportunities that would fit my profile. I would be thrilled to discuss about those with you."
}
```
{
    "message": "Hello Adriel, as a graduate from an MSc in Statistics with

In [85]:
answer

NameError: name 'answer' is not defined

# Work to get optimzed messages

In [19]:
import openai

In [20]:
openai.api_key = open_file('open_ai_key.txt')[2:-2]

In [21]:
mess_quant = "graduate from an MSc in Statistics and Quantitative Finance. An experience as a data scientist, looking for an opportunity as a quant"
mess_data = "graduate from an MSc in Statistics. An experience as a data scientist, looking for a new opportunity, data-driven."

In [22]:
def gpt3_chat(message: str, messages:list[dict])-> tuple[str, list]:
    '''Returns GPT's response to the input message and appends it to the messages list'''
    messages.append({"role": "user", "content": message})
    # ChatGPT is powered by gpt-3.5-turbo, OpenAI’s most advanced language model.
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)['choices'][0]['message']['content']
    messages.append({"role": "assistant", "content": response})
    return str(response), messages

In [23]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

test = test_api.get_profile(search_quant[7]['public_id'])
test['summary'][0:500]

In [102]:
discussion = [
        {"role": "system", "content": "You are a helpful assistant that personalize generic message asking for a connection on linkedin to headhunters from their summary in no more than 300 character."},
        {"role": "user", "content": f"Here is the generic message : <<{mess}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{test['summary']}>>"},
        {"role": "assistant", "content": "What is the expected output format?"},        
]


message_prompt = "No more than 240 character. Just the motivation part, no intro or outro. Don't say hello. Do it now."
answer, discussion = gpt3_chat(message=message_prompt, messages=discussion)
print(answer)
answers.append([answer, discussion])

"Your extensive experience in financial services as a specialist executive recruiter caught my attention. I would greatly appreciate connecting with you to discuss potential opportunities as a quant. Thank you for your time."


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant that personalize generic message asking for a connection on linkedin to headhunters from their summary."},
        {"role": "user", "content": f"Here is the generic message : <<{mess}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{prof['summary']}>>"},
        {"role": "assistant", "content": "What is the expected output format?"},        
    ]
)

experience_prompt = "A motivation message in no more than 50 words to invite the headhunter to contact me if he sees a fit. Do it now."
experience, cv_discussion = utils.gpt3_chat(message=experience_prompt, messages=cv_discussion)
print(experience)

# Work to find optimal job posts

- When an interesting job is found, we want to connect with the recruiter but we can't get his profile from the get_job() func already implemented
- We will try to add a func to the class linkedin to get this data

In [209]:
test_job = test_api.test_get_job(jobs[0]['entityUrn'].split(":")[3])

in test_job


In [20]:
test = test_api.search_jobs(keywords = 'Quant quantitative researcher',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'London, England')

In [12]:
test = test_api.search_jobs(keywords = 'Data Scientist ML AI', title = 'Data Scientist',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'London, England')

In [429]:
test_job = test_api.search_jobs(keywords = 'quantitative researcher', experience = "2", 
                                limit = 100, listed_at = 14 * 24 * 60 * 60)

In [170]:
jobs = test_api.search_jobs(keywords = 'data', limit = 5)

# Work with the package selenium to fetch data from recruiter

In [22]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import unittest


In [23]:
def connect_firefox_to_my_linkedin(credentials):
    
    browser = webdriver.Firefox()

    # your secret credentials:
    email = credentials[0]
    password = credentials[1]
    # Go to linkedin and login
    browser.get('https://www.linkedin.com/login')
    time.sleep(3)
    browser.find_element(by =By.ID, value ='username').send_keys(email)
    browser.find_element(by =By.ID, value ='password').send_keys(password)
    browser.find_element(by =By.ID, value ='password').send_keys(Keys.RETURN)
    
    time.sleep(3)
    
    return browser


def get_recruiter_id_from_job(browser, job_id):
    
    browser.get(f"https://www.linkedin.com/jobs/view/{job_id}")

    time.sleep(2)
    
    sele_search = browser.find_elements(by=By.CLASS_NAME, value="mh4.pt4.pb3")
    
    if(len(sele_search) == 0):
        return False
    
    recruiter_public_id = sele_search[0].find_element(by=By.CSS_SELECTOR,value ='a').get_attribute('href').split('/')[-1]
    
    return recruiter_public_id

In [24]:
browser = connect_firefox_to_my_linkedin(test_credential)
get_id = get_recruiter_id_from_job(browser, "3583408428")

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:457:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1525:15
despatch@chrome://remote/content/marionette/server.sys.mjs:302:40
execute@chrome://remote/content/marionette/server.sys.mjs:275:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:248:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:249:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:496:20


# Work to link our message to a collab sheet

https://towardsdatascience.com/turn-google-sheets-into-your-own-database-with-python-4aa0b4360ce7

In [11]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# attributes that can be of use :
#    - sheet.get_all_values()
#    - sheet.row_values(n) n:int

In [12]:
def add_new_row(sheet, row_to_insert):
    
    current_number_rows = len(sheet.get_all_values())
    sheet.insert_row(row_to_insert, current_number_rows + 1)

    return True

def delete_last_row(sheet):
    
    current_number_rows = len(sheet.get_all_values())
    sheet.delete_rows(current_number_rows)
    
    return True


def create_sheet(sheet_name, mail):
    
    scope = ['https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive"]

    credentials = ServiceAccountCredentials.from_json_keyfile_name("gs_credentials.json", scope)
    client = gspread.authorize(credentials)

    sheet = client.create(sheet_name)
    sheet.share(mail, perm_type='user', role='writer')
    
    return sheet


def connect_to_sheet(sheet_name):
    
    scope = ['https://www.googleapis.com/auth/spreadsheets',
            "https://www.googleapis.com/auth/drive"]

    credentials = ServiceAccountCredentials.from_json_keyfile_name("gs_credentials.json", scope)
    client = gspread.authorize(credentials)
    
    sheet = client.open(sheet_name).sheet1
    
    return sheet

In [33]:
### Only create the sheet once, if it has been deleted, use this command

#mail = "tang.bellec@gmail.com"
#sheet = create_sheet("messages_sent", mail)
#sheet = connect_to_sheet(sheet_name)
#add_new_row(sheet, ["job_id", "recruiter_id", "message"])

True

In [28]:
sheet = connect_to_sheet("messages_sent")

# Pipeline to dm recruiters from job post

https://www.linkedin.com/mynetwork/invitation-manager/sent/ link to see invite sent

### extract job title

In [9]:
import json

def json_to_dict(json_string):
    """Converts a JSON-formatted string to a Python dictionary."""
    return json.loads(json_string)

def gpt_extract_job_function_from_title(job_title):

    prompt = f"""
    Your task is to extract the job title from a job post title. \
    We just want to keep the job function, no more than 3 words.
    Provide it in JSON format with the key: job_title 

    Job post title: ```{job_title}```
    """

    response = get_completion(prompt)
    response = json_to_dict(response)
    
    return response


def add_easy_connection_from_recruiter_job_post(recruiter_id, api):
    
    message = ""
    
    current_recruiter = api.get_profile(recruiter_id)

    first_name = get_first_name_fetched_profile(current_recruiter)
    print(first_name)
    
    is_french = is_profile_fetched_french(current_recruiter)
    print(is_french)
    
    sector = is_profile_data_or_finance(current_recruiter)
    print(sector)

    if(is_french):
        print("french")
        
    else:
        
        if(sector == 'finance'):
            message = "Hello " + first_name + "\n" + message_quant + "\n" + end_of_message

        else:
            message = "Hello " + first_name + "\n" + message_data + "\n" + end_of_message


    connection = api.add_connection(current_recruiter['public_id'], message)
    
    return [sector, connection]


def create_message_for_recruiter_from_job(recruiter_profile_id, job_title, api = test_api):
    
    recruiter = api.get_profile(recruiter_profile_id)
    first_name = recruiter['firstName']
    
    if(first_name):
        
        message = "Hello "+ first_name + ", \n" + "I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    else:
        
        message = "Hello, I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    
    return message

In [47]:
jobs = test_api.search_jobs(keywords = 'Data Scientist', title = 'Data Scientist',
                                limit = 100, listed_at = 7 * 24 * 60 * 60, location_name = 'London, England')

browser = connect_firefox_to_my_linkedin(true_credential)

In [50]:
no_id = 0

for job in jobs[5:100]:
    
    current_job_id = job['entityUrn'].split(":")[3]
    recruiter_id = get_recruiter_id_from_job(browser, current_job_id)
    
    if(recruiter_id):
        
        job_title = job['title']
        job_title = gpt_extract_job_function_from_title(job_title)['job_title']
        
        message = create_message_for_recruiter_from_job(recruiter_id, job_title)
        
        connection = real_api.add_connection(recruiter_id, message)
        
        add_new_row(sheet, [current_job_id, recruiter_id, message])
    
    else:
        print("can't get id")
        no_id += 1
        
    time.sleep(2)
    
    
    
        

can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
can't get id
